# Popular Optimizers

Optimizers are implemented based on book Deep Learning.

Gradients are computed automatically by MXNet

In [ ]:
import mxnet as mx

## First Order Method
1. Stochastic Gradient Descent
2. SGD with Momentum
3. Nesterov Accelerated Gradient Descent

In [ ]:
def sgd(params, batch_size, lr):
    for param in params:
        param[:] = param - lr * param.grad

def sgd_momentum(params, prev_state, lr, batch_size, momentum=.9):
    for param in params:
        state = -lr * param.grad / batch_size + momentum * prev_state
        param[:] = param + state
        
def nesterov(params, state, lr, batch_size, momentum=.9):
    for param in params:
        state = -lr * param.grad / batch_size + momentum * prev_state
        param[:] += -momentum * prev_state + (1 + momentum) * state

## Adaptive Methods

1. AdaGrad
2. RMSProp
3. Adam

In [ ]:
def adagrad(params, lr, batch_size, delta=1e-7):
    # init gradient accumulation
    r = 0
    def update(params, lr, batch_size):
        for param in params:
            # compute gradient
            g = param.grad / batch_size
            # accumulate
            r += mx.nd.square(g)
            # compute update
            adagrad = (lr / (delta + mx.nd.sqrt(r))) * g
            # apply update
            param[:] += -adagrad
    return update(params, lr, batch_size)

def rmsprop(params, lr, wd, batch_size, delta=1e-6):
    # init gradient accumulation
    r = 0
    def update(params, lr, batch_size):
        for param in params:
            # compute gradient
            g = param.grad / batch_size
            # accumulate
            r = wd * r + (1 - wd) * mx.nd.square(g)
            # compute update
            rmsgrad = (lr / (delta + mx.nd.sqrt(r))) * g
            # apply update
            param[:] += -rmsgrad
    return update(params, lr, batch_size)

def adam(params, batch_size, 
         delta=1e-8, rho_1=.9, rho_2=.999, eps=1e-3):
    # init gradient accumulation
    s = 0
    r = 0
    t = 0
    def update(batch_size, delta, rho_1, rho_2, eps):
        for param in params:
            # compute gradient
            g = param.grad / batch_size
            # update time step
            t += 1
            # update biased first moment
            s = rho_1 * s + (1 - rho_1) * g
            # update biased second moment
            r = rho_2 * r + (1 - rho_2) * mx.nd.square(g)
            # correct bias in 1st moment
            s = s / (1 - rho_1 ** t)
            # correct bias in 2nd moment
            r = r / (1 - rho_2 ** t)
            # compute update
            adamgrad = eps * (s / (delta + mx.nd.sqrt(r)))
            # apply update
            param[:] += -adamgrad
    return update(batch_size, delta, rho_1, rho_2, eps)

## Comments

Generally, adaptive methods can be viewed as second order moment approxiamations.

Adaptive methods require less babysitting while their performance on test set may be not as good as SGD.